In [1]:
!pip install kafka-python

In [2]:
from kafka import KafkaConsumer
import json
import uuid
import random
import time
import cProfile
import csv
import os

In [4]:
servers= ['broker1:9093', 'broker2:9094', 'broker3:9095']
topic1 = "products"

consumer = KafkaConsumer(
    topic1, 
    auto_offset_reset='earliest',
    bootstrap_servers=servers, 
    api_version=(0, 10), 
    value_deserializer = json.loads,
    consumer_timeout_ms=1000
)

In [5]:
def data_processor(consumer):
    output = []
    for msg in consumer:
        product = msg.value.get("product")
        product["code"] = random.randint(400, 500)
        output.append(product)
    return output

In [6]:
consume_stats = [
    ["consumed_messages", "time_needed"]
]
def consume(consumer, consume_stats):
    consume = 0
    msgs = []
    start = time.time()
    for msg in consumer:
        msgs.append(msg.value.get("product"))
        consume += 1
    end = time.time()
    time_needed = end - start
    consume_stats.append([consume, time_needed])
    return consume_stats

In [7]:
# changed_data = data_processor(consumer)
# cProfile.run("get_changed_data(consumer)")

In [8]:
# def write_json_to_file(filepath, content):
#     with open(filepath, "w") as file:
#         json.dump(content, file)

In [9]:
# write_json_to_file("mc1-part1/changed_data.json", changed_data)

In [ ]:
with open("stats.csv", "w", newline="") as file:
    consumed_stats = consume(consumer, consume_stats)
    writer = csv.writer(file)
    writer.writerows(consumed_stats)